# PETs/TETs – Hyperledger Aries / PySyft – City (Relying Party) 🏙️

In [1]:
%%javascript
document.title ='🏙️ City Agent'

<IPython.core.display.Javascript object>

## PART 3: Connect with Manufacturers and Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* Manufacturer1 💼: `03_connect_with_city.ipynb`
* Manufacturer2 💼: `03_connect_with_city.ipynb`
* Manufacturer3 💼: `03_connect_with_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import libs.helpers as helpers
import os
import syft as sy
import time
import torch

from aries_cloudcontroller import AriesAgentController
from libs.agent_connection_manager import RelyingParty
from pprintpp import pprint
from termcolor import colored
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate City Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
city_agent = RelyingParty(agent_controller)

Successfully initiated AgentConnectionManager for a(n) RelyingParty ACA-PY agent


---

<a id=2></a>

### 2 – Establish a connection with the Authority agent

#### 2.1  – Create multi-use invitation
Send out a multi-use invitation to all manufacturer agents (i.e., copy & paste the same invitation to manufacturer1, manufacturer2, manufacturer3). This represents a scenario where the City agent invites any agent to connect with them, and authenticate as manufacturers. 

🕵️‍♂️ The advantage of a multi-use invitation is, that the City is unaware about who accessed the invitation link and is trying to get in contact with the City. The City agent only knows the names defined in `ACAPY_LABEL` (see respective `.env` files), and does not know which agent is which manufacturer.

**Note:** Please establish a connection with all three manufacturer agents.

In [7]:
# Setup for connection with Authority agent
alias = None
auto_accept = True # Accept response of Authority agent right away
auto_ping = True
public = False # Do not use public DID
multi_use = True # Invitation is only for one invitee

invitation = city_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, auto_ping=auto_ping)


Copy & paste invitation and share with external agent(s):
{
    '@id': 'ac1eee2e-c63e-4415-a8a7-7ce5b434ea0f',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City-Agency',
    'recipientKeys': ['9wRd53Mahf746UnxEJgTDnyyRwYogTWEnxRFUSAj1LFA'],
    'serviceEndpoint': 'https://0a6f916f7909.ngrok.io',
}

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  29b4bcd4-317a-492f-b5e5-99d352755c1f
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9bffebd1-205b-44be-8f8f-99cc61c378f2
State :  invitation (invitation-sent)
Routing State : none
Their Role :  invitee
------------------------------------------------------------

<div style="font-size: 25px"><center><b>Break Point 1</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 💼💼💼</center></div><br>
<center><b>Please open all manufacturer agents 💼💼💼. <br> For each of the manufacturer agents, open the 03_connect_with_city.ipynb notebook and execute all steps until Break Point 2/3/4. <br> Use the same invitation from the City Agent 🏙️ in Step 2.1 for all Manufacturers.</b></center>


#### 2.2 – Display all active connections

In [8]:
# Display all active connections
for conn in city_agent.get_active_connections():
    conn.display()


---------------------------------------------------------------------
Connection with AnonymousAgent1
Connection ID :  9bffebd1-205b-44be-8f8f-99cc61c378f2
Connection with :  AnonymousAgent1
Is Active :  True
Connection Alias :  None
Presentation Exchange IDs :  []
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection with AnonymousAgent2
Connection ID :  1186e299-c470-4c92-b164-5c582f3f3f47
Connection with :  AnonymousAgent2
Is Active :  True
Connection Alias :  None
Presentation Exchange IDs :  []
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection with AnonymousAgent3
Connection ID :  55427664-4e76-4a4a-bcca-1693e9e2da38
Connection with :  AnonymousAgent3
Is Active :  True
Connection Alias :  None
Presentation Exchange IDs :  []
-----------------------------------------------------------

#### 2.3 – Fetch all connection_ids of the active connections with the respective manufacturer agents

In [9]:
# Get conneciton_id with Manufacturer1
connection_id_a1 = city_agent.get_connection_id("AnonymousAgent1")[0] # We assume that there is only one connection with Manufacturer1

# Get conneciton_id with Manufacturer2
connection_id_a2 = city_agent.get_connection_id("AnonymousAgent2")[0] # We assume that there is only one connection with Manufacturer2

# Get conneciton_id with Manufacturer3
connection_id_a3 = city_agent.get_connection_id("AnonymousAgent3")[0] # We assume that there is only one connection with Manufacturer3

### 3 – Send Proof Request

#### 3.1 Define VC Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections requesting that they produce a presentation containing the identified attributes and meeting the restrictions.

It is often useful to define your request objects first, then reuse these objects across many connections you wish to request a proof from. 

Duplicate and customise the below cell as many times as you need. It may be useful to save these request objects either to the jupyter store using %store or through

TODO: Detail the full set of restrictions available to a verifier.

In [10]:
# Define which scheme we want a proof from
identifiers = helpers.get_identifiers()
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]

# Define the attributes we want from the scheme
req_attrs = [{"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id}]}]

# Define proof_request
manufacturer_proof_request = {
    "name": "isManufacturer Proof Request",
    "version": "1.0",
    "requested_attributes": {f"0_{req_attr['name']}_uuid": req_attr for req_attr in req_attrs },
    # Ignore predicates (e.g., range proofs)
    "requested_predicates": {}, #{f"0_{req_pred['name']}_GE_uuid": req_pred for req_pred in req_preds },
    # You can also request the entire proof request be non-revoked
    "non_revoked":  {"to": int(time.time())}
}

print(colored("Manufacturer Proof Request:", attrs=["bold"]))
pprint(manufacturer_proof_request)

Get stored identifiers dictionary to access schema information 💾
Manufacturer Proof Request:
{
    'name': 'isManufacturer Proof Request',
    'non_revoked': {'to': 1629369989},
    'requested_attributes': {
        '0_isManufacturer_uuid': {
            'name': 'isManufacturer',
            'restrictions': [
                {
                    'schema_id': 'XvWddwCGryjykggMo3vHrv:2:certify-manufacturer:0.0.1',
                },
            ],
        },
    },
    'requested_predicates': {},
    'version': '1.0',
}


#### 3.2 – Send Proof Request
The proof request asks the agent at the other end of `connection_id` to prove attributes defined within `manufacturer_proof_request` using the manufacturer schema.

The resulting presentation request is encoded in base64 and packed into a DIDComm Message. The `@type` attribute in the presentation request defines the protocol present-proof and the message request-presentation.

Overall, the proof-presentation proceedure has xx steps:
1. request_sent
2. request_received
3. presentation_sent
4. presentation_received
5. presentation_acked 
6. verified

In [11]:
# Send proof request to Manufacturer1 and get presentation_exchange_id for connection with manufacturer 1
presentation_exchange_id_a1 = city_agent.send_proof_request(
    connection_id=connection_id_a1, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)


# Send proof request to Manufacturer2 and get presentation_exchange_id for connection with manufacturer 2
presentation_exchange_id_a2 = city_agent.send_proof_request(
    connection_id=connection_id_a2, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)

# Send proof request to Manufacturer3 and get presentation_exchange_id for connection with manufacturer 3
presentation_exchange_id_a3 = city_agent.send_proof_request(
    connection_id=connection_id_a3, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)


---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  9bffebd1-205b-44be-8f8f-99cc61c378f2
Presentation Exchange ID :  472782a6-2f84-41a1-8e18-69e29e0ad65b
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self
---------------------------------------------------------------------

Presentation Request : 
{
    '@id': 'c30589b8-cf8f-4854-9d62-810559bb5f44',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation',
    'comment': 'Please prove that you are an agent who is a certified manufacturer',
    'request_presentations~attach': [
        {
            '@id': 'libindy-request-presentation-0',
            'data': {
                'base64': 'eyJuYW1lIjogImlzTWFudWZhY3R1cmVyIFByb29mIFJlcXVlc3QiLCAidmVyc2lvbiI6ICIxLjAiLCAicmVxdWVzdGVkX2F0dHJpYnV0ZXMiOiB7IjBfaXNNYW51ZmFjdHVyZXJfdXVpZCI6IHsibmFtZSI6ICJpc01hbnVmYWN0dXJlciIsICJyZXN0cmljdGlvbnMiOiBbeyJ

<div style="font-size: 25px"><center><b>Break Point 5</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 💼💼💼</center></div><br>
<center><b>Please go to all manufacturer agents 💼💼💼. <br> For each of the manufacturer agents, continue with executing Step 3</b></center>

#### 3.3 – Verify Proof Presentations

When proof presentations are received by the manufacturers (see output under Step 3.2), you can verify whether the proof presentations are valid. 
A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

Execute the following cell. The `verify_proof_presentation()` function verifies the proof presentation, and parses out relevant information. The returning value states whether the presentation proof is valid or not. In the use case at hand, because only one variable is verified, the boolean denotes if the agent is a manufacturer or not.

In [13]:
a1_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a1)
a2_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a2)
a3_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a3)


---------------------------------------------------------------------
Presentation Exchange ID 472782a6-2f84-41a1-8e18-69e29e0ad65b
Presentation valid :  True
Revealed Attributes : 
	* isManufacturer = TRUE
Self-Attested Attributes : 
Predicate Attributes : 
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  9bffebd1-205b-44be-8f8f-99cc61c378f2
Presentation Exchange ID :  472782a6-2f84-41a1-8e18-69e29e0ad65b
Protocol State :  verified
Agent Role :  verifier
Initiator :  self
---------------------------------------------------------------------
Presentation Exchange ID: 472782a6-2f84-41a1-8e18-69e29e0ad65b is verified

---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  1186e299-c470-4c92-b164-5c582f3f3f47
Presentation Exchange ID :  

--- 

### 4 – Do Data Science

#### 4.1 – Establish a Duet connection
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [13]:
# Step 4: Connect to duet of M1
duet_m1 = sy.duet("86a6992e42ce977bbe6e9bf698d25a72")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: e8e53fdde51e445cc96272ea8145f101

♫♫♫ > ...waiting for partner to connect...


KeyboardInterrupt: 

In [ ]:
print(f"duet id: {duet_m1.id}, duet target it: {duet_m1.target_id}, duet note: {duet_m1.node}")

In [ ]:
duet_m1.store.pandas

In [ ]:
from IPython.core.debugger import set_trace

duet_m1.store.pandas

## 2 - Secure Multi-Party Computation

In [ ]:
# Step 5: Setup session with all duets
session = Session(parties=[duet_m1])
print(session)

In [ ]:
# Step 6: Setup MPC session
SessionManager.setup_mpc(session)

In [ ]:
#duet_ds.store.pandas

In [ ]:
#from IPython.core.debugger import set_trace

# Step 7: NOW we're ready to set up private operaitions
#%pdb
duet_m1.store.pandas

In [ ]:
# Step 12: basic operations
x_secret = duet_m1.store["test_data_age"] # describe local data to test sum, substract, and multiply

In [ ]:
y = torch.Tensor([[-5,8,1,7,6,100]]) # Local data

In [ ]:
x = MPCTensor(secret=x_secret, shape=(1,), session=session)  # MPC Tensor from x_secret

The following lines of codes always receive an error message.
[See Github issue](https://github.com/OpenMined/SyMPC/issues/282).

In [ ]:
print("X + Y = ",(x + y).reconstruct())

In [ ]:
print("X - Y = ", (x - y).reconstruct())

In [ ]:
print("X * Y = ", (x * y).reconstruct())

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()